In [139]:
import numpy as np, re
from itertools import cycle
from aoclib import list2string, string2list
from itertools import combinations

In [173]:
test_string("b")

('b', False)

In [195]:
def test_string(msg, ruleno = 0):
    current_rule = rules[ruleno]

    if msg == '':
        return msg, False
    
    if current_rule in [['a'], ['b']]:
        if msg[0] == current_rule[0]:
            if len(msg) > 1:
                return msg[1:], True
            else:
                return '', True
        else:
            return msg, False
            
    if type(current_rule) == list:

        msg_sav = msg
        for r in current_rule:
            msg, success = test_string(msg, r)
            if not success:
                return msg_sav, success

        if success:
            return msg, success
                
    elif type(current_rule) == tuple:
        
        msg_sav = msg
        for r in current_rule[0]:
            msg, success = test_string(msg, r)
            if not success:
                break
                
        if success:
            return msg, success
            
        msg = msg_sav
        for r in current_rule[1]:
            msg, success = test_string(msg, r)
            if not success:
                return msg_sav, success

        if success:
            return msg, success

def test_string_wrapper(msg, ruleno = 0):
    msg, success = test_string(msg, ruleno = ruleno)
    if len(msg) > 0:
        return False
    else:
        return success

In [196]:
# Part 1
lines = open('19.txt').read().strip()

rules_input, messages_input = lines.split("\n\n")
rules = dict()
for i, ri in enumerate(rules_input.splitlines()):
    no, subrules = ri.split(":")
    parts = subrules.split("|")
    if len(parts) == 2:
        rules[int(no)] = ([int(x) for x in parts[0].split()], [int(x) for x in parts[1].split()])
    else:
        rules[int(no)] = ([int(x) if x.isnumeric() else x[1] for x in parts[0].split()])
        
total = 0
for message in messages_input.splitlines():
    res = test_string_wrapper(message)
    total += res
print(total)

187


In [323]:
import copy

# Part 2
lines = open('19.txt').read().strip()

rules_input, messages_input = lines.split("\n\n")
rules = dict()
for i, ri in enumerate(rules_input.splitlines()):
    no, subrules = ri.split(":")
    parts = subrules.split("|")
    if len(parts) == 2:
        rules[int(no)] = ([int(x) for x in parts[0].split()], [int(x) for x in parts[1].split()])
    else:
        rules[int(no)] = [int(x) if x.isnumeric() else x[1] for x in parts[0].split()]
rules[8] = tuple((rules[8], [42, 8]))
rules[11] = tuple((rules[11], [42, 11, 31]))

#8: 42 | 42 8
#11: 42 31 | 42 11 31 
        
total = 0
yy = []
for message in messages_input.splitlines():
    res = test_string_wrapper(message)
    if res == False:
        x1 = test_string(message, ruleno = 42)
        x1 = test_string(x1[0], ruleno = 42)
        residual = copy.deepcopy(x1)
        it = 0
        while residual[1] == True:
            it += 1
            residual = test_string(residual[0], ruleno = 42)
        flag = 1
        for i in range(1, it):
            x2 = copy.deepcopy(x1)
            for _ in range(i):
                #print(j)
                #break
                x2 = test_string(x2[0], ruleno = 42)
            for j in range(i + 2):
                x2 = test_string(x2[0], ruleno = 31)
                if x2[1] == True and x2[0] == '':
                        if message == 'abbbabaabbbaabaaabbbbbaaabbabababababbbabaabbabaaabababbbbbbbbbbbbbbabba':
                            print(message)
                            print(x1[0])
                            print("i", i, "j", j)
                        res = True
                        flag = 0
                        #print(message)
                        break
                if x2[1] == False:
                    break
            if flag == 0:
                break
        #print(it, "max iteration")
    
    total += res
    yy.append((res, message))
print(total)

392


In [262]:
x = test_string('aabbaabb', ruleno= 31)
x

('', True)

In [263]:
test_string('abbbbaaabbbbaabbabbbaabaababbaababbbabbb', ruleno=42)
isValid(getRule(42), 'abbbbaaabbbbaabbabbbaabaababbaababbbabbb')

False

In [322]:
x = test_string('abbbabaabbbaabaaabbbbbaaabbabababababbbabaabbabaaabababbbbbbbbbbbbbbabba', ruleno=42)
for i in range(3):
    x = test_string(x[0], ruleno = 42)
x

('bababbbabaabbabaaabababbbbbbbbbbbbbbabba', True)

In [304]:
y = test_string('bababbbabaabbabaaabababbbbbbbbbbbbbbabba', ruleno=42)
print(len(y), y)

2 ('bababbbabaabbabaaabababbbbbbbbbbbbbbabba', False)


In [317]:
z = test_string('bbbbabba', ruleno=31)
print(len(z), z)

2 ('', True)


In [267]:
for i in range(len(y)):
    if y[i][-1] != yy[i][0]:
        print(y[i], yy[i])

('abbbabaabbbaabaaabbbbbaaabbabababababbbabaabbabaaabababbbbbbbbbbbbbbabba', False) (True, 'abbbabaabbbaabaaabbbbbaaabbabababababbbabaabbabaaabababbbbbbbbbbbbbbabba')


In [266]:
def makeRules(fn):
  with open(fn) as file:
    raw = file.read().split("\n\n")
  rulesDict = {}
  for rule in raw[0].split("\n"):
    rule = rule.split(":")
    if "\"" in rule[1]:
      subrules = rule[1][2]
    elif "|" in rule[1]:
      subrules = [[int(j)for j in i.strip().split()] for i in rule[1].strip().split("|")]
    else:
      subrules = [int(i) for i in rule[1].strip().split(" ")]
    rulesDict[int(rule[0])] = subrules
  messages = raw[1].split("\n")

  return rulesDict, messages

def printRules(rulesDict):
  print("Printing Rules")
  for rule in rulesDict:
    print(str(rule) + ": " + str(rulesDict[rule]))
  print("Done.")

#Takes a formatted password string and calculates the length.
#Assumes all branches of the password are uniform in length.
#Input: "a[[aa|bb][ab|ba]|[ab|ba][aa|bb]]b"
#Output: 6
def getLen(rule): #Computes the length of password rules for isvalid.
  cur, count = 0, 0
  while cur < len(rule):
    if rule[cur] in ("a", "b"):
      count += 1
    elif rule[cur] == "|":
      depth = 1
      while depth > 0:
        cur += 1
        if rule[cur] == "|":
          depth += 1
        elif rule[cur] == "]":
          depth -= 1
    cur += 1

  return count

#Determines if a given message is valid against the formatted pasword string
#Inputs: (String) formatted password string, (String) message
#Output: Boolean 
def isValid(rule, message):
  ptrs = [0]
  mPtr = [0]
  ruleLen = getLen(rule)

  if len(message) != ruleLen:
    return False

  #Because we know that the length of the message matches the check exactly we know that once you get past the nth letter, you've passed.
  while ptrs:
    if mPtr[0] >= len(message):
      return True

    cur = rule[ptrs[0]] #Get the value pointed to by the current pointer
    if cur.isalpha(): #If it's a simple a or b, evaluate it. If it's bad, trash the pointers and start from the next one.
      if message[mPtr[0]] == cur:
        ptrs[0] += 1
        mPtr[0] += 1
      else:
        ptrs.pop(0)
        mPtr.pop(0)
    elif cur == "[": #Create a second pointer after the relevant "|". Watch for errant "["s. Increment pointer.
      tempPtr, depth = ptrs[0] + 1, 1
      while depth > 0:
        if rule[tempPtr] == "[":
          depth += 1
        elif rule[tempPtr] == "|":
          depth -= 1
        tempPtr += 1
      ptrs.append(tempPtr)
      mPtr.append(mPtr[0])
      ptrs[0] += 1
    elif cur == "|": # Skip forward to after the relevant "]". Watch for errant "["s.
      depth = 1
      while depth > 0:
        ptrs[0] += 1
        if rule[ptrs[0]] == "[":
          depth += 1
        elif rule[ptrs[0]] == "]":
          depth -= 1
      ptrs[0] += 1
    else: #Cur is at a "]", just increment it.
      ptrs[0] += 1

  #If you're run out of valid pointers to check, the rule must be invalid.
  return False

#Takes a set of rules and creates a formatter password string.
#Input: [4, 1, 5]
#Output: "a[[aa|bb][ab|ba]|[ab|ba][aa|bb]]b"
def getRule(ruleIdx):
  global rulesDict
  rule = rulesDict[ruleIdx]
  while not all(type(x) == str for x in rule):
    i = 0
    if type(rule[0]) == list:
        rule = ["["] + rule[0] + ["|"] + rule[1] + ["]"]

    while i < len(rule):
      instruction = rule[i]
      #print(str(i) + ": " + str(instruction))
      if type(instruction) == str:
        pass
      else:
        newInst = rulesDict[instruction]
        if type(newInst) == int or type(newInst) == str:
          rule = rule[:i] + [newInst] + rule[i + 1:]
        else:
          if type(newInst[0]) == int:
            rule = rule[:i] + newInst + rule[i + 1:]
          else:
            rule = rule[:i] + ["["] + newInst[0] + ["|"] + newInst[1] + ["]"] + rule[i + 1:]
      i += 1

  rule = "".join(rule)
  return rule

global y
y = []
#Check message segments against rule 42 twice, then check it against 42 until it fails, then check it against 31 until it fails or you pass.
def part2():
  rule42, rule31 = getRule(42), getRule(31)
  ruleLen = getLen(rule42)
  count = 0

  for message in messages:
    old_count = count
    if len(message) % ruleLen != 0 or len(message) < (ruleLen * 3):
      pass #Don't even bother validating.
    elif len(message) == (ruleLen * 3):
      count += 1 if isValid(rule42 + rule42 + rule31, message) else 0
    else:
      if isValid(rule42, message[0:ruleLen]) and isValid(rule42, message[ruleLen:2*ruleLen]) and isValid(rule31, message[-ruleLen:]):
        midSegs = int(len(message) / ruleLen) - 3
        valid42 = all(isValid(rule42, message[(2+i)*ruleLen:(3+i)*ruleLen]) for i in range(int(midSegs/2) + midSegs%2))

        flag = 42
        for i in range(int(midSegs/2), 0, -1):
          if flag == 42: 
            flag = 42 if isValid(rule42, message[(i+1)*-ruleLen:i*-ruleLen]) else 31
          if flag == 31:
            flag = 31 if isValid(rule31, message[(i+1)*-ruleLen:i*-ruleLen]) else 0
        count += 1 if flag > 0 and valid42 else 0
        
      else:
        pass
    if count != old_count:
        y.append((message, True))
    else:
        y.append((message, False))
  return count


rulesDict, messages = makeRules("19.txt")
rule0 = getRule(0)
print(getLen(rule0))
#total = sum(isValid(rule0, message) for message in messages)
#print("Final count: " + str(total))

p2count = part2()
print("Final count: " + str(p2count))

24
Final count: 392
